figment-fusion-logo.svg

# Figment Fusion

`v1.0.0-alpha.4`

This is a pre-release version of Figment Fusion. You are welcome to share your feedback and contribute on [GitHub](https://github.com/rlaneth/figment-fusion).

---

Hello there! 👋

You've arrived at Figment Fusion, your way into the world of AI-based image generation.

This notebook is designed to provide insights on the process of running [Stable Diffusion](https://stability.ai/blog/stable-diffusion-announcement), allowing for exploration and customization while remaining simple and fun to use.

Stable Diffusion is a state-of-the-art, free and open text-to-image model created by researchers from [CompVis](https://github.com/CompVis) and [Runway](https://runwayml.com/), with support from [Eleuther AI](https://www.eleuther.ai/), [LAION](https://laion.ai/) and [Stability AI](https://stability.ai/).

The [Diffusers library](https://github.com/huggingface/diffusers) by Hugging Face is used here with inspiration from the _[Stable Diffusion with Diffusers](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb)_ demo.

## 🌱 Getting Started

The steps in this notebook should be completed in the order they appear. Executing them in the wrong order might result in errors or other unexpected behavior.

The following code snippet displays the GPUs available on your current runtime, assisting you in determining whether you are working in an appropriate environment.

In [ ]:
!nvidia-smi -L

If you are running this notebook on hosted [Google Colab](https://colab.research.google.com/) runtime, you might be provided with either a NVIDIA Tesla T4 or a P100, two of the most frequently seen of the GPU models made available through the service. As according to the [Google Colab FAQ](https://research.google.com/colaboratory/faq.html), the available GPU types may vary over time and access to resources is never guaranteed.

**Note:** previous versions of this notebook incorrectly stated that the Tesla P100 was faster and offered more VRAM than the T4. Both provide the same amount of VRAM, and data suggests that the T4 may be slightly faster than the P100 when generating images with Figment Fusion.

## ⚙️ Settings

In [ ]:
#@title Model Selection

MODEL_MAP = {
  'FP16 Stable Diffusion v1.1': ['CompVis/stable-diffusion-v1-1', 'fp16'],
  'FP16 Stable Diffusion v1.2': ['CompVis/stable-diffusion-v1-2', 'fp16'],
  'FP16 Stable Diffusion v1.3': ['CompVis/stable-diffusion-v1-3', 'fp16'],
  'FP16 Stable Diffusion v1.4': ['CompVis/stable-diffusion-v1-4', 'fp16'],
  'FP16 Waifu Diffusion': ['hakurei/waifu-diffusion', 'fp16'],
  'Stable Diffusion v1.1': ['CompVis/stable-diffusion-v1-1', 'main'],
  'Stable Diffusion v1.2': ['CompVis/stable-diffusion-v1-2', 'main'],
  'Stable Diffusion v1.3': ['CompVis/stable-diffusion-v1-3', 'main'],
  'Stable Diffusion v1.4': ['CompVis/stable-diffusion-v1-4', 'main'],
  'Waifu Diffusion': ['hakurei/waifu-diffusion', 'main']
}

#@markdown You can choose which model to use for image generation here. Consider the following
#@markdown points for making your choice.

#@markdown - Half precision (i.e. `fp16`) variants of each model require less storage space (around
#@markdown   2.6 GB) compared to the full precision versions (around 5.2 GB), and consume less
#@markdown   [VRAM](https://www.techtarget.com/searchstorage/definition/video-RAM).
#@markdown - Older versions of Stable Diffusion are supported, but the latest one is generally
#@markdown    better.
#@markdown - [Waifu Diffusion](https://huggingface.co/hakurei/waifu-diffusion) is a model based on
#@markdown   Stable Diffusion v1.4 and fine-tuned for anime style art.

#@markdown To download each model, you must have an account on the Hugging Face website and accept
#@markdown the terms on the page of the repository that correponds to the model you've chosen. For
#@markdown example, in order to retrieve the Stable Diffusion v1.4 model (and its `fp16` variant),
#@markdown you are required to accept the terms on
#@markdown [CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4).

use_model = "FP16 Stable Diffusion v1.4" #@param ["FP16 Stable Diffusion v1.1", "FP16 Stable Diffusion v1.2", "FP16 Stable Diffusion v1.3", "FP16 Stable Diffusion v1.4", "FP16 Waifu Diffusion", "Stable Diffusion v1.1", "Stable Diffusion v1.2", "Stable Diffusion v1.3", "Stable Diffusion v1.4", "Waifu Diffusion"]

model_clone_repository = MODEL_MAP[use_model][0]
model_clone_branch = MODEL_MAP[use_model][1]

In [ ]:
#@title Safety Checker

#@markdown By default, if the generation model creates content which is determined to be unsafe
#@markdown (e.g. sexually explicit), it will not be displayed or saved, being replaced with a black
#@markdown image instead. Use this parameter to toggle this feature.

enable_safety_checker = True #@param { "type": "boolean" }

In [ ]:
#@title Storage

#@markdown The value of `storage_mount_path` is set to the the mount point of Google Drive on the
#@markdown Google Colab environment. If you are running on a different environment, you should
#@markdown change this parameter to specify a suitable location for data storage (e.g. the root
#@markdown of a persistent volume mounted to a virtual machine).
storage_mount_path = '/content/drive/MyDrive' #@param { "type": "string" }

#@markdown The `storage_data_path` parameter specifies a directory to store all data related to
#@markdown Figment Fusion, including the model cache and generated images. It is relative to
#@markdown `storage_mount_path`, meaning those two values are concatenated to obtain the full path.
storage_data_path = '/Colab Data/Figment Fusion' #@param { "type": "string" }

storage_output_path = '/output'
storage_cache_path = '/cache'

storage_data_full_path = storage_mount_path + storage_data_path
storage_output_full_path = storage_data_full_path + storage_output_path
storage_cache_full_path = storage_data_full_path + storage_cache_path

## 👮 Permissions

In [ ]:
#@title Google Drive { vertical-output: true }

#@markdown If you are running on Google Colab, the notebook requires permission to access your
#@markdown Google Drive. For other environments, you should set `storage_mount_path` accordingly and
#@markdown ignore this step.

try:
  from pathlib import Path
  from google.colab import drive
  drive_mount_path = storage_mount_path.rstrip('MyDrive')
  Path(drive_mount_path).mkdir(parents=True, exist_ok=True)
  drive.mount(drive_mount_path, force_remount=True)
except ImportError:
  print('Not running on Google Colab')

In [ ]:
#@title Hugging Face { vertical-output: true }

#@markdown This step requires an access token to your Hugging Face account, which will be used to
#@markdown retrieve the selected generation model. You can safely ignore it if you have already
#@markdown downloaded the model before and wish for it to be loaded from the cache.

!pip install --quiet huggingface_hub
!git config --global credential.helper store

try:
  from google.colab import output
  output.enable_custom_widget_manager()
except ImportError:
  pass

from huggingface_hub import notebook_login
notebook_login()

## 📦 Requirements

In [ ]:
#@title Dependencies { vertical-output: true }
#@markdown Ensures the presence of required Python packages and data storage directories.

from pathlib import Path
!pip install --quiet diffusers==0.3.0 transformers scipy ftfy 'ipywidgets>=7,<8'
Path(storage_output_full_path).mkdir(parents=True, exist_ok=True)
Path(storage_cache_full_path).mkdir(parents=True, exist_ok=True)

In [ ]:
#@title Pipeline { vertical-output: true }
#@markdown Creates the Stable Diffusion pipeline. If the generation model is not present in the
#@markdown cache, downloading will also be performed in this step.

import torch
from diffusers import StableDiffusionPipeline
from huggingface_hub import notebook_login, whoami

if model_clone_branch == 'fp16':
  print('Selected half precision model')
  torch_dtype = torch.float16
else:
  print('Selected full precision model')
  torch_dtype = torch.float32

try:
  whoami()
  print('Hugging Face authentication token is present')
  local_files_only = False
except ValueError as e:
  print('Hugging Face authentication token not present')
  print('Model will be loaded from cache')
  local_files_only = True

pipe = StableDiffusionPipeline.from_pretrained(
  model_clone_repository,
  revision=model_clone_branch,
  torch_dtype=torch_dtype,
  cache_dir=storage_cache_full_path,
  local_files_only=local_files_only,
  use_auth_token=(not local_files_only)
)

if not enable_safety_checker:
  def safety_checker(images, **kwargs):
    return images, False
  pipe.safety_checker = safety_checker

pipe = pipe.to('cuda')

In [ ]:
#@title Helpers
#@markdown Defines auxiliary constants, functions and classes required by Figment Fusion.

import json
import pprint
import time
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler
from PIL import Image

# Constants

STORAGE_META_PATH = '/meta'
FF_DISPLAY_NAME = 'Figment Fusion'
FF_VERSION = 'v1.0.0-alpha.4'

# Functions

def get_output_meta(batch_meta):
  return {
    'app': FF_DISPLAY_NAME,
    'version': FF_VERSION,
    'model': use_model,
    'batch': batch_meta
  }

def save_images(base_path, images, current_batch, meta):
  meta_path = '{}/{}/{}.json'.format(base_path, STORAGE_META_PATH, current_batch)
  meta_detailed = get_output_meta(meta)
  meta_serialized = json.dumps(meta_detailed, indent=2)
  with open(meta_path, 'w') as f:
    f.write(meta_serialized)  
  for i, image in enumerate(images):
    image_path = '{}/{}-{}.png'.format(base_path, current_batch, i)
    image.save(image_path)

def display_images_as_grid(images):
  columns = len(images)
  width, height = images[0].size
  grid = Image.new('RGB', size=(columns * width, height))
  grid_width, grid_height = grid.size
  for i, image in enumerate(images):
    grid.paste(image, box=(i % columns * width, i // columns * height))
  display(grid)

def display_images(images, as_grid = False):
  if as_grid:
    display_images_as_grid(images)
    return
  for image in images:
    display(image)

def display_meta(meta):
  pprint.pprint(meta)

# Classes

class Scheduler():
  @property
  def scheduler(self):
    return self.__scheduler

  def __init__(self, name, *args, **kwargs):
    self.name = name
    self.beta_start = kwargs.get('beta_start', 0.00085)
    self.beta_end = kwargs.get('beta_end', 0.012)
    self.beta_schedule = kwargs.get('beta_schedule', 'scaled_linear')
    self.ddim_clip_sample = kwargs.get('ddim_clip_sample', False)
    self.ddim_set_alpha_to_one = kwargs.get('ddim_set_alpha_to_one', False)
    self.ddim_eta = kwargs.get('ddim_eta', 0)

    scheduler_class = PNDMScheduler

    if name == 'DDIM':
      self.__scheduler = DDIMScheduler(
        beta_start=self.beta_start,
        beta_end=self.beta_end,
        beta_schedule=self.beta_schedule,
        clip_sample=self.ddim_clip_sample,
        set_alpha_to_one=self.ddim_set_alpha_to_one
      )
      return

    if name == 'LMS':
      scheduler_class = LMSDiscreteScheduler

    self.__scheduler = scheduler_class(
      beta_start=self.beta_start,
      beta_end=self.beta_end,
      beta_schedule=self.beta_schedule
    )

class Generator():
  def __init__(self, pipe, *args, **kwargs):
    self.pipe = pipe
    self.width = kwargs.get('width', 512)
    self.height = kwargs.get('height', 512)
    self.num_inference_steps = kwargs.get('num_inference_steps', 50)
    self.guidance_scale = kwargs.get('guidance_scale', 7.5)
    self.manual_seed = kwargs.get('manual_seed', None)
    self.__generator = torch.Generator('cuda')
    self.__scheduler = kwargs.get('scheduler', None)
    if self.__scheduler == None:
      self.__scheduler = Scheduler('PNDM')
  
  @property
  def scheduler(self):
    return self.__scheduler
  
  def scheduler(self, value):
    self.pipe.register_modules(scheduler=value)
    self.__scheduler = value

  def generate_latents(self, batch_size, offset):
    size_offset = batch_size + offset
    latents_shape = (size_offset, self.pipe.unet.in_channels, self.height // 8, self.width // 8)
    latents = torch.randn(
      latents_shape,
      generator=self.__generator,
      device='cuda'
    )
    return latents[offset:, ...]
  
  def run(self, text_prompt, images_per_batch = 1, skip_images_batch = 0):
    pipe = self.pipe
    generator = self.__generator
    generator_seed = self.manual_seed
    width = self.width
    height = self.height
    num_inference_steps = self.num_inference_steps
    guidance_scale = self.guidance_scale
    scheduler = self.__scheduler
    ddim_eta = scheduler.ddim_eta

    if generator_seed == None:
      generator_seed = generator.seed()
    else:
      generator.manual_seed(generator_seed)
    
    latents = self.generate_latents(images_per_batch, skip_images_batch)

    if isinstance(text_prompt, str):
      prompt = [text_prompt]
      display_prompt = prompt

    if images_per_batch > 1:
      prompt = prompt * images_per_batch
      display_prompt = text_prompt

    meta = {
      'prompt': display_prompt,
      'width': width,
      'height': height,
      'num_inference_steps': num_inference_steps,
      'guidance_scale': guidance_scale,
      'scheduler': {
        'name': scheduler.name,
        'beta_start': scheduler.beta_start,
        'beta_end': scheduler.beta_end,
        'beta_schedule': scheduler.beta_schedule
      },
      'seed': generator_seed,
      'num_images': len(prompt)
    }

    if scheduler.name == 'DDIM':
      meta['scheduler']['ddim_clip_sample'] = scheduler.ddim_clip_sample
      meta['scheduler']['ddim_set_alpha_to_one'] = scheduler.ddim_clip_sample
      meta['scheduler']['ddim_eta'] = ddim_eta
    
    with torch.autocast('cuda'):
      images = pipe(
        prompt,
        width=width,
        height=height,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        latents=latents,
        eta=ddim_eta
      ).images

    return (images, meta)

## 🎨 Usage

In [ ]:
#@markdown ### General

text_prompt = '' #@param { "type": "string" }
width = 512 #@param {type:"slider", min:128, max:1024, step:128}
height = 512 #@param {type:"slider", min:128, max:1024, step:128}
num_inference_steps = 50 #@param {type:"slider", min:1, max:100, step:1}
guidance_scale = 7.5 #@param {type:"slider", min:0, max:30, step:0.5}

#@markdown ### Seed

#@markdown The `manual_seed` parameter is ignored if `use_random_seed` is enabled.
use_random_seed = True #@param { "type": "boolean" }
manual_seed = 0 #@param {type:"integer"}

#@markdown ### Scheduler

#@markdown Parameters prefixed with `ddim_` are only used for the DDIM scheduler.
use_scheduler = 'PNDM' #@param ["PNDM", "DDIM", "LMS"]
beta_start = 0.00085 #@param {type:"number"}
beta_end = 0.012 #@param {type:"number"}
beta_schedule = 'scaled_linear' #@param ["linear", "scaled_linear", "squaredcos_cap_v2"]
ddim_clip_sample = False #@param { "type": "boolean" }
ddim_set_alpha_to_one = False #@param { "type": "boolean" }
ddim_eta = 0 #@param {type:"slider", min:0, max:1, step:0.00001}

#@markdown ### Batch

num_batches = 1 #@param { "type": "number" }
images_per_batch = 1 #@param { type: "slider", min: 1, max: 4, step: 1 }
skip_images_batch = 0 #@param { type: "slider", min: 0, max: 4, step: 1 }

#@markdown ### Mutation

mutate_seed = True #@param { type: "boolean" }
mutate_guidance_scale = False #@param { type: "boolean" }
mutate_guidance_scale_min = 7.5 #@param {type:"slider", min:0, max:30, step:0.5}
mutate_guidance_scale_max = 21.5 #@param {type:"slider", min:0, max:30, step:0.5}
mutate_guidance_scale_interval = 1 #@param {type:"slider", min:0, max:5, step:0.5}

#@markdown ### Output

display_output = True #@param { "type": "boolean" }
display_batch_as_grid = True #@param { "type": "boolean" }
display_output_meta = True #@param { "type": "boolean" }
save_output = True #@param { "type": "boolean" }
save_output_path = '/batch_{timestamp}' #@param { "type": "string" }

timestamp_start = time.time()
print('Starting up ...')

if save_output:
  timestamp_output = str(int(timestamp_start))
  storage_batch_path = save_output_path.replace('{timestamp}', timestamp_output)
  storage_batch_full_path = storage_output_full_path + storage_batch_path
  storage_meta_full_path = storage_batch_full_path + STORAGE_META_PATH
  Path(storage_meta_full_path).mkdir(parents=True, exist_ok=False)

if use_random_seed:
  manual_seed = None

scheduler = Scheduler(
  use_scheduler,
  beta_start=beta_start,
  beta_end=beta_end,
  beta_schedule=beta_schedule,
  ddim_clip_sample=ddim_clip_sample,
  ddim_set_alpha_to_one=ddim_set_alpha_to_one,
  ddim_eta=ddim_eta
)

generator = Generator(
  pipe,
  width=width,
  height=height,
  num_inference_steps=num_inference_steps,
  guidance_scale=guidance_scale,
  manual_seed=manual_seed,
  scheduler=scheduler
)

for i in range(num_batches):
  if i > 0:
    if not mutate_seed:
      generator.manual_seed = meta['seed']
    if mutate_guidance_scale:
      guidance_scale = meta['guidance_scale'] + mutate_guidance_scale_interval
      if guidance_scale > mutate_guidance_scale_max:
        guidance_scale = mutate_guidance_scale_min
      generator.guidance_scale = guidance_scale
  
  current_batch = i + 1
  print('Batch {} of {}'.format(current_batch, num_batches))

  images, meta = generator.run(text_prompt, images_per_batch, skip_images_batch)
  
  if display_output:
    display_images(images, display_batch_as_grid)
  if display_output_meta:
    display_meta(meta)
  if save_output:
    save_images(storage_batch_full_path, images, current_batch, meta)

timestamp_end = time.time()
execution_time = timestamp_end - timestamp_start

print('Finished ({} seconds)'.format(execution_time))